In [1]:
import tensorflow as tf

In [2]:
c_0 = tf.constant(0, name="c")  # => operation named "c"

# Already-used names will be "uniquified".
c_1 = tf.constant(2, name="c")  # => operation named "c_1"

# Name scopes add a prefix to all operations created in the same context.
with tf.name_scope("outer"):
  c_2 = tf.constant(2, name="c")  # => operation named "outer/c"

  # Name scopes nest like paths in a hierarchical file system.
  with tf.name_scope("inner"):
    c_3 = tf.constant(3, name="c")  # => operation named "outer/inner/c"

  # Exiting a name scope context will return to the previous prefix.
  c_4 = tf.constant(4, name="c")  # => operation named "outer/c_1"

  # Already-used name scopes will be "uniquified".
  with tf.name_scope("inner"):
    c_5 = tf.constant(5, name="c")  # => operation named "outer/inner_1/c"

In [3]:
"""
# Operations created outside either context will run on the "best possible"
# device. For example, if you have a GPU and a CPU available, and the operation
# has a GPU implementation, TensorFlow will choose the GPU.
weights = tf.random_normal(...)

with tf.device("/device:CPU:0"):
  # Operations created in this context will be pinned to the CPU.
  img = tf.decode_jpeg(tf.read_file("img.jpg"))

with tf.device("/device:GPU:0"):
  # Operations created in this context will be pinned to the GPU.
  result = tf.matmul(weights, img)
"""

'\n# Operations created outside either context will run on the "best possible"\n# device. For example, if you have a GPU and a CPU available, and the operation\n# has a GPU implementation, TensorFlow will choose the GPU.\nweights = tf.random_normal(...)\n\nwith tf.device("/device:CPU:0"):\n  # Operations created in this context will be pinned to the CPU.\n  img = tf.decode_jpeg(tf.read_file("img.jpg"))\n\nwith tf.device("/device:GPU:0"):\n  # Operations created in this context will be pinned to the GPU.\n  result = tf.matmul(weights, img)\n'

In [4]:
"""
# Create a default in-process session.
with tf.Session() as sess:
  # ...
"""

'\n# Create a default in-process session.\nwith tf.Session() as sess:\n  # ...\n'

In [5]:
x = tf.constant([[37.0, -23.0], [1.0, 4.0]])
w = tf.Variable(tf.random_uniform([2, 2]))
y = tf.matmul(x, w)
output = tf.nn.softmax(y)
init_op = w.initializer

with tf.Session() as sess:
  # Run the initializer on `w`.
  sess.run(init_op)

  # Evaluate `output`. `sess.run(output)` will return a NumPy array containing
  # the result of the computation.
  print(sess.run(output))

  # Evaluate `y` and `output`. Note that `y` will only be computed once, and its
  # result used both to return `y_val` and as an input to the `tf.nn.softmax()`
  # op. Both `y_val` and `output_val` will be NumPy arrays.
  y_val, output_val = sess.run([y, output])

[[  9.99998569e-01   1.39670942e-06]
 [  5.24527371e-01   4.75472569e-01]]


** Graphs and sessions **

In [6]:
g_1 = tf.Graph()
with g_1.as_default():
  # Operations created in this scope will be added to `g_1`.
  c = tf.constant("Node in g_1")

  # Sessions created in this scope will run operations from `g_1`.
  sess_1 = tf.Session()

g_2 = tf.Graph()
with g_2.as_default():
  # Operations created in this scope will be added to `g_2`.
  d = tf.constant("Node in g_2")

# Alternatively, you can pass a graph when constructing a `tf.Session`:
# `sess_2` will run operations from `g_2`.
sess_2 = tf.Session(graph=g_2)

assert c.graph is g_1
assert sess_1.graph is g_1

assert d.graph is g_2
assert sess_2.graph is g_2

In [7]:
# Print all of the operations in the default graph.
g = tf.get_default_graph()
print(g.get_operations())

[<tf.Operation 'c' type=Const>, <tf.Operation 'c_1' type=Const>, <tf.Operation 'outer/c' type=Const>, <tf.Operation 'outer/inner/c' type=Const>, <tf.Operation 'outer/c_1' type=Const>, <tf.Operation 'outer/inner_1/c' type=Const>, <tf.Operation 'Const' type=Const>, <tf.Operation 'random_uniform/shape' type=Const>, <tf.Operation 'random_uniform/min' type=Const>, <tf.Operation 'random_uniform/max' type=Const>, <tf.Operation 'random_uniform/RandomUniform' type=RandomUniform>, <tf.Operation 'random_uniform/sub' type=Sub>, <tf.Operation 'random_uniform/mul' type=Mul>, <tf.Operation 'random_uniform' type=Add>, <tf.Operation 'Variable' type=VariableV2>, <tf.Operation 'Variable/Assign' type=Assign>, <tf.Operation 'Variable/read' type=Identity>, <tf.Operation 'MatMul' type=MatMul>, <tf.Operation 'Softmax' type=Softmax>]
